In [ ]:
import zipfile as zip

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset

import sys
sys.path.insert(0,'../')
from common.datautils import load_ce_results
from common.datautils import format_data_for_multilabel
from common.featuresutils import load_features
from common.flagutils import load_flag_list
from common.flagutils import load_o3_flags
from common.flagutils import build_config
from common.flagutils import get_cmd_string_from_config
from common.flagutils import remove_static_labels

In [ ]:
average_data = load_ce_results('CE.results.zip')
average_data.head()

In [ ]:
all_flags = load_flag_list()
print(len(all_flags))

In [ ]:
def best_configuration(variable, benchmark, average_data):
    benchmark_data = average_data.loc[average_data["Benchmark"] == benchmark]
    min_index = benchmark_data[variable].idxmin()
    return benchmark_data.loc[min_index]["Flags"]


def get_o3_config():
    return get_cmd_string_from_config(build_config(all_flags, load_o3_flags(), '-O3'))

In [ ]:
benchmarks = average_data["Benchmark"].unique()
benchmarks_nosize = [b.split('.')[0] for b in benchmarks]
print(benchmarks_nosize)

In [ ]:
def load_data(with_dwarf):
    X = StandardScaler().fit_transform(load_features(benchmarks_nosize, with_dwarf=with_dwarf))
    y = []

    for benchmark in benchmarks:
        config_str = best_configuration('Energy', benchmark, average_data)
        if config_str == '-O3':
            config_str = get_o3_config()

        labels = []

        config = config_str[4:].split(' ')
        for i, flag in enumerate(config):
            if flag == all_flags[i]:
                labels.append(1) # Flag is turned on
            elif flag == '-fno-' + all_flags[i][2:]:
                labels.append(0) # FLag is turned off -fno
            else:
                print("ERROR:" + flag)

        y.append(labels)

    return X, np.array(y)


def grid_search(classifier, parameters, X, y):
    gs = GridSearchCV(classifier, parameters, cv=15, scoring='f1_macro')
    gs.fit(X, y)

    return gs


def test_multilabel_classifier(classifier, X, y):
    classifier_name = type(classifier).__name__
    
    print(f"Testing {classifier_name}")
    
    parameters = [
        {
            'classifier': [SVC(kernel='linear')],
            'classifier__C': np.logspace(-2, 10, 13)
        },
        {
            'classifier': [SVC(kernel='rbf')],
            'classifier__C': np.logspace(-2, 10, 13), 
            'classifier__gamma':  np.logspace(-9, 3, 13)
        },
        {
            'classifier': [GaussianNB(), DecisionTreeClassifier(max_depth=5), AdaBoostClassifier(), DecisionTreeClassifier(max_depth=5)]
        }
    ]
    gs = grid_search(classifier, parameters, X, y)

    print(f"{classifier_name} Best F1 Score: {gs.best_score_}")
    print(gs.best_params_)
    print("---------")    

In [ ]:
def run_comparison(multilabel_classifiers, disable_warnings=False, with_dwarf=False):
    X, y = load_data(with_dwarf)
    y = remove_static_labels(y)

    print(f"Loaded {X.shape[0]} samples with {X.shape[1]} features and {y.shape[1]} labels.")
    
    # Grid search can throw a lot of warnings. Is useful to just mute them sometimes.
#     if disable_warnings:
#         import warnings
#         warnings.filterwarnings('ignore')
        
    for clf in multilabel_classifiers:
        test_multilabel_classifier(clf, X, y)

In [ ]:
#run_comparison([BinaryRelevance(), ClassifierChain(), LabelPowerset()], disable_warnings=True, with_dwarf=False)

In [ ]:
#run_comparison([BinaryRelevance(), ClassifierChain(), LabelPowerset()], disable_warnings=True, with_dwarf=True)

In [ ]:
X, y = format_data_for_multilabel(average_data, False, benchmarks)
y, remaining_labels = remove_static_labels(y, with_labels=True)

kf = KFold(n_splits=len(y))
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = ClassifierChain(SVC(kernel='rbf', C=0.01, gamma=0.01))
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test).toarray()

    config = ""

    for index, flag_enabled in enumerate(prediction[0]):
        if flag_enabled:
            config += remaining_labels[0][index] + " "
        else:
            config += "-fno-" + remaining_labels[0][index] + " "

    print(benchmarks[test_index])
    print(config)
    print("-----")
